<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://raw.githubusercontent.com/Arize-ai/phoenix-assets/9e6101d95936f4bd4d390efc9ce646dc6937fb2d/images/socal/github-large-banner-phoenix.jpg" width="1000"/>
        <br>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Tracing CrewAI with Arize Phoenix - Parallelization Workflow</h1>

In [ ]:
!pip install -q arize-phoenix opentelemetry-sdk opentelemetry-exporter-otlp crewai crewai_tools openinference-instrumentation-crewai

# Set up Keys and Dependencies

Note: For this colab you'll need:

*   OpenAI API key (https://openai.com/)
*   Serper API key (https://serper.dev/)
*   Phoenix API key (https://app.phoenix.arize.com/)

In [ ]:
import getpass
import os

# Prompt the user for their API keys if they haven't been set
openai_key = os.getenv("OPENAI_API_KEY", "OPENAI_API_KEY")
serper_key = os.getenv("SERPER_API_KEY", "SERPER_API_KEY")

if openai_key == "OPENAI_API_KEY":
    openai_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")

if serper_key == "SERPER_API_KEY":
    serper_key = getpass.getpass("Please enter your SERPER_API_KEY: ")

# Set the environment variables with the provided keys
os.environ["OPENAI_API_KEY"] = openai_key
os.environ["SERPER_API_KEY"] = serper_key

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass.getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

## Configure Tracing

In [ ]:
from phoenix.otel import register

tracer_provider = register(
    project_name="crewai-agents", endpoint="https://app.phoenix.arize.com/v1/traces"
)

# Instrument CrewAI

In [ ]:
from openinference.instrumentation.crewai import CrewAIInstrumentor

CrewAIInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)

## Define your Agents

In [ ]:
from crewai import Agent, Crew, Task
from crewai.process import Process

researcher_1 = Agent(
    role="LLM Researcher A",
    goal="Research trend #1 in AI and summarize it clearly.",
    backstory="Specializes in model safety and governance.",
    verbose=True,
)

researcher_2 = Agent(
    role="LLM Researcher B",
    goal="Research trend #2 in AI and summarize it clearly.",
    backstory="Expert in multimodal and frontier models.",
    verbose=True,
)

researcher_3 = Agent(
    role="LLM Researcher C",
    goal="Research trend #3 in AI and summarize it clearly.",
    backstory="Focused on AI policy and alignment.",
    verbose=True,
)

aggregator = Agent(
    role="Aggregator",
    goal="Combine and synthesize all research into a single summary report.",
    backstory="Information architect skilled at summarizing multiple sources.",
    verbose=True,
)

## Define your Tasks

In [ ]:
# Define parallel research tasks
task1 = Task(
    description="Summarize a major trend in AI safety and model alignment.",
    expected_output="Concise summary of trend #1",
    agent=researcher_1,
)

task2 = Task(
    description="Summarize a key innovation in multimodal or frontier AI systems.",
    expected_output="Concise summary of trend #2",
    agent=researcher_2,
)

task3 = Task(
    description="Summarize a current topic in AI policy, regulation, or social impact.",
    expected_output="Concise summary of trend #3",
    agent=researcher_3,
)

# Aggregation task
aggregation_task = Task(
    description="Combine the three AI trend summaries into a cohesive single report.",
    expected_output="A synthesized report capturing all three trends.",
    agent=aggregator,
)

## Create Crew

In [ ]:
crew = Crew(
    agents=[researcher_1, researcher_2, researcher_3, aggregator],
    tasks=[task1, task2, task3, aggregation_task],
    process=Process.sequential,
    verbose=True,
)

result = crew.kickoff()
print(result)

### Check your Phoenix project to view the traces and spans from your runs.